<a href="https://colab.research.google.com/github/cathyxinchangli/ATMS-597-SP-2020/blob/master/ATMS-597-SP-2020-Project-2/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import numpy as np
import pandas as pd
import datetime

import matplotlib as mlp
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates

%matplotlib inline

In [2]:
def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    ---
    S. Moline, 'Hands on Data Analysis with Pandas'
    https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/blob/master/ch_04/0-weather_data_collection.ipynb
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'HfZphTTGbXckpmEuqevAXQZFRCTEJgkJ' # obtained by X. Li, 2/4/2020
        },
        params=payload
    )

def fetch_data(locationid, startdate, enddate, endpoint="data",
               datasetid="GHCND", stationid=None, units="metric", limit=100):
    response = make_request(
        endpoint,
        {
            "datasetid": datasetid,
            "datatypeid": ["TMAX", "TMIN"],
            "locationid": locationid,
            "stationid": stationid,
            "startdate": startdate,
            "enddate": enddate,
            "unit": units,
            "limit": limit,
        })
    response = pd.DataFrame(response.json()["results"])
    return response

def clean_up(datain, key):
    '''
    S. Nesbitt
    '''
    datain[key][(np.abs(datain[key])==9999)] = None
    datain[key] = datain[key].astype("float") / 10.
    datain["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)
    datain.index = datain["date"]
    return datain
    

In [3]:
df = fetch_data("CITY:US360019", datetime.date(2017, 1, 1), datetime.date(2017, 4, 1))
df = clean_up(df, "value")
df.describe()
df.head()

C:\Users\YangLu\Anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,datatype,station,attributes,value
date,,,,,
2017-01-01,2017-01-01,TMAX,GHCND:USC00280907,",,7,0700",5.6
2017-01-01,2017-01-01,TMIN,GHCND:USC00280907,",,7,0700",-2.8
2017-01-01,2017-01-01,TMAX,GHCND:USC00281335,",,7,0700",10.0
2017-01-01,2017-01-01,TMIN,GHCND:USC00281335,",,7,0700",-2.2
2017-01-01,2017-01-01,TMAX,GHCND:USC00283704,",,7,0700",6.7


In [4]:
def stripe_plot(df, freq="W"):
    df_mean = df.resample(freq).mean()
    if freq == "W":
        date_interval = 7
    
    temp_min = df_mean.loc[:, "value"].min()
    temp_max = df_mean.loc[:, "value"].max()
    buffer = 1.
    
    cmap = ListedColormap([
        '#08306b', '#08519c', '#2171b5', '#4292c6',
        '#6baed6', '#9ecae1', '#c6dbef', '#deebf7',
        '#fee0d2', '#fcbba1', '#fc9272', '#fb6a4a',
        '#ef3b2c', '#cb181d', '#a50f15', '#67000d',
    ])
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_axes([0.1, 0.12, 0.9, 0.88])
    date_str_list = ['{}'.format(d) for d in df['date'][0:730:2]]
    date_corrected = mdates.datestr2num(date_str_list)

    col = PatchCollection([
        Rectangle((y, min_temp-buffer), 7, temp_delta+2*buffer) # need to change 7 to other values when resampled at other frequencies
        for y in date_corrected.astype(int)[::7]
    ])